In [1]:
from lsst.rsp import get_tap_service, retrieve_query
import pandas as pd
import random

In [2]:
service = get_tap_service("tap")

In [3]:
filename = '/scratch/DouglasLTucker/dp02_dc2_catalogs_Object_all_objectids_sorted.csv'
random.seed(4321)
df = pd.read_csv(filename, 
           skiprows=lambda x: x > 0 and random.random() >=0.00001)

In [4]:
print(len(df))

2872


The use of ``WHERE <col> IN ()`` is a temporary work-around for DP0.2 until the functionality for user-uploaded tables
(and joins with user-uploaded tables) is ready.

In [5]:
# N = len(df)
N = 100
string_list = '('
for i in range(N):
    string_list = string_list + str(df['objectId'][i])
    if i < N-1:
        string_list = string_list + ','
    elif i == N-1:
        string_list = string_list + ')'

In [6]:
query = '''SELECT obj.objectId, obj.detect_isPrimary 
           FROM dp02_dc2_catalogs.Object AS obj 
           WHERE obj.objectId IN ''' + string_list

Synchronous queries are not recommended and will not work for queries that take >30 min.

In [7]:
# results = service.search(query).to_table()

Run an asynchronous query.

In [8]:
job = service.submit_job(query)
print('Job URL is', job.url)
print('Job phase is', job.phase)

Job URL is https://data.lsst.cloud/api/tap/async/s6bpo72zbazyxwyo
Job phase is PENDING


In [9]:
job.run()

Check on the job phase. It will say "EXECUTING" while the job is in progress, and "COMPLETED" when it is done.

At this time, there is no way to check on the progress or to have a time estimate, or to get a notificiation when it is done.

In [13]:
print('Job phase is now ', job.phase)

Job phase is now  COMPLETED


In [14]:
async_results = job.fetch_result().to_table().to_pandas()

In [ ]:
async_results